In [38]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

from fuzzywuzzy import fuzz


# Load all six dataframes 
df_bechdel = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/Movie Bechdel Test Scores/Bechdel.csv')
df1 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/FINAL DATASETS SELECTION/2Mmovies.csv')
df2 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/FINAL DATASETS SELECTION/100k_Movies_dataset.csv')
df3 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/clean_for_modelling_fin.csv')
df4 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/additional datasets/25k IMDb movie Dataset.csv')




/var/folders/k8/l3hrl_fn5y7bx5qcfv9y8p900000gn/T/ipykernel_58399/1013081592.py:18: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/FINAL DATASETS SELECTION/2Mmovies.csv')


In [39]:
df_bechdel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9373 non-null   int64  
 1   title       9373 non-null   object 
 2   imdbid      9373 non-null   float64
 3   id          9373 non-null   int64  
 4   rating      9373 non-null   int64  
 5   year        9373 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 439.5+ KB


In [40]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590932 entries, 0 to 2590931
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              object 
 1   name            object 
 2   year            object 
 3   rating          float64
 4   certificate     object 
 5   duration        object 
 6   genre           object 
 7   votes           object 
 8   gross_income    object 
 9   directors_id    object 
 10  directors_name  object 
 11  stars_id        object 
 12  stars_name      object 
 13  description     object 
dtypes: float64(1), object(13)
memory usage: 276.7+ MB


In [41]:

df1 = df1.rename(columns={'id': 'imdbid'})
df1['imdbid'] = df1['imdbid'].str[2:]
df1['imdbid'] = df1['imdbid'].astype('float64')
df1['imdbid']

0          4710316.0
1          1281857.0
2          2294954.0
3          6843596.0
4          3705604.0
             ...    
2590927    9898738.0
2590928    9898742.0
2590929    9898746.0
2590930    9898802.0
2590931    9898812.0
Name: imdbid, Length: 2590932, dtype: float64

In [42]:
df2 = df2.rename(columns={'id': 'imdbid'})
df2['imdbid'] = df2['imdbid'].str[2:]
df2['imdbid'] = df2['imdbid'].astype('float64')


In [43]:
df4
df4 = df4.rename(columns={'path': 'imdbid'})
df4['imdbid'] = df4['imdbid'].str[9:]
df4['imdbid'] = df4['imdbid'].str[:-1]
df4['imdbid'] = df4['imdbid'].astype('float64')

In [44]:
# A left join will include all rows from the left (or "first") dataframe
#(i.e. the original dataframe), and any matching rows from the right (or "additional") dataframe. 
#If there are any missing values in the right dataframe, the corresponding columns in the merged dataframe will contain null values.
merged_df_left = pd.merge(df_bechdel, df1, on='imdbid', how='left')
merged_df_left = pd.merge(merged_df_left, df2, on='imdbid', how='left')
merged_df_left = pd.merge(merged_df_left, df3, on='imdbid', how='left')
merged_df_left = pd.merge(merged_df_left, df4, on='imdbid', how='left')

/var/folders/k8/l3hrl_fn5y7bx5qcfv9y8p900000gn/T/ipykernel_58399/3136487250.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'year_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df_left = pd.merge(merged_df_left, df4, on='imdbid', how='left')


In [46]:
merged_df_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9496 entries, 0 to 9495
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      9496 non-null   int64  
 1   title_x         9496 non-null   object 
 2   imdbid          9496 non-null   float64
 3   id              9496 non-null   int64  
 4   rating_x        9496 non-null   int64  
 5   year_x          9496 non-null   int64  
 6   name            6694 non-null   object 
 7   year_y          6694 non-null   object 
 8   rating_y        6694 non-null   float64
 9   certificate_x   6694 non-null   object 
 10  duration        6694 non-null   object 
 11  genre_x         6694 non-null   object 
 12  votes_x         6694 non-null   object 
 13  gross_income    6694 non-null   object 
 14  directors_id    6694 non-null   object 
 15  directors_name  6694 non-null   object 
 16  stars_id        6694 non-null   object 
 17  stars_name      6694 non-null   o

In [47]:
# Check for duplicate movie imdbid in the final dataframe
duplicates = merged_df_left.duplicated(subset='imdbid')

# Count the number of duplicates
num_duplicates = sum(duplicates)

# Print the number of duplicates
print("Number of duplicate movie titles: ", num_duplicates)

Number of duplicate movie titles:  135


In [48]:
# Remove rows with duplicate movie titles
df_matches_dropped = merged_df_left.drop_duplicates(subset='imdbid', keep=False)

# Check the new number of rows in the dataframe
print("Number of rows in the cleaned dataframe: ", len(df_matches_dropped))

Number of rows in the cleaned dataframe:  9231


In [49]:
df_matches_dropped

,Unnamed: 0,title_x,imdbid,id,rating_x,year_x,name,year_y,rating_y,certificate_x,...,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year_y
0,0,Passage de Venus,3155794.0,9602,0,1874,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,La Rosace Magique,14495706.0,9804,0,1877,La Rosace Magique,(1877),5.9,Not Certified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sallie Gardner at a Gallop,2221420.0,9603,0,1878,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Le singe musicien,12592084.0,9806,0,1878,Le singe musicien,(1878),6.2,Not Certified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Athlete Swinging a Pick,7816420.0,9816,0,1881,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,9367,Encanto,2953050.0,10151,3,2021,魔法滿屋,(2021),7.2,PG,...,1 hour 42 minutes,7.2,203K,"['Animation', 'Comedy', 'Family']",A Colombian teenage girl has to face the frustration of being the only member of her family without magical powers.,"['magic', '3d animation', 'colombia', 'house', 'forest', 'cg animation', 'pop musical', 'magical house', 'toucan', 'symbolism']",Jared Bush,"['Charise Castro Smith', 'Charise Castro Smith', 'Jared Bush', 'Byron Howard', 'Stephanie Beatriz']",Byron Howard,-2021
9491,9368,Love Hard,10752004.0,10152,2,2021,NaN,NaN,NaN,NaN,...,not-released,6.3,50K,"['Comedy', 'Romance']","An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted romantic comedy chronicles her attempt to reel in love.","['christmas', 'travel', 'hometown', 'love', 'friend', 'journalist', 'guy', 'joke', 'social media', 'holiday']",Hernan Jimenez,"['Rebecca Ewing', 'Nina Dobrev', 'Jimmy O. Yang', 'Darren Barnet', 'Hernan Jimenez']",Daniel Mackey,-2021
9493,9370,West Side Story,3581652.0,10157,3,2021,West Side Story,(2021),7.2,PG-13,...,"$100,000,000 (estimated)",7.2,74K,"['Crime', 'Drama', 'Musical']","An adaptation of the 1957 musical, West Side Story explores forbidden love and the rivalry between the Jets and the Sharks, two teenage street gangs of different ethnic backgrounds.","['remake', 'gang', 'based on stage musical', ""reference to shakespeare's romeo and juliet"", 'racism', 'new york city', 'love', 'dying in arms', 'dance', 'singing']",Steven Spielberg,"['Arthur Laurents', 'Ansel Elgort', 'Rachel Zegler', 'Ariana DeBose', 'Steven Spielberg']",Tony Kushner,-2021
9494,9371,Every Time a Bell Rings,15943414.0,10158,3,2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Reset the index of df_matches_dropped
# df_matches_dropped = df_matches_dropped.reset_index(drop=True)
# # create separate columns for each genre, rename and merge with original dataframe
# # genres = df_matches_dropped['genre'].str.split(',', expand=True)
# # genres.columns = ['genre_1', 'genre_2', 'genre_3']
# # df_matches_dropped = pd.concat([df_matches_dropped, genres], axis=1)

# #drop the not needed columns
# df_matches_dropped = df_matches_dropped.drop(['title', 'bechdel_id', 'bechdel_score', 'year_x', 'certificate_y', 'runtime_min', 'genre', 'imdb_rating', 'about', 'director_y', 'stars', 'votes_y', 'gross_earn', 'genre_1', 'genre_2', 'genre_3'], axis=1)

# #rename ambiguously named columns 
# # df = df_matches_dropped.rename(columns={'id': 'bechdel_id', 'rating_x': 'bechdel_score', 'rating_y': 'imdb_rating'})



In [55]:
# all data for title, , id, rating_x (bechdel), year  so remove the other columns 
df_matches_dropped = df_matches_dropped.drop(['title_y', 'title', 'movie title', 'bechdel_id', 'bechdel_score', 'year_y', 'released_year', 'year_x', 'year_y'], axis=1)


# # create new column year by combining year_x and year_y
# df_matches_dropped['film_title'] = merged_df['title_x'].fillna(merged_df['year_y']).astype(int)

# # drop year_x and year_y columns
# merged_df = merged_df.drop(['year_x', 'year_y'], axis=1)

In [57]:
df_matches_dropped = df_matches_dropped.drop(['name', 'genre_1','genre_2', 'genre_3'], axis=1)

In [ ]:
df_matches_dropped = df_matches_dropped.drop(['name', 'genre_1','genre_2', 'genre_3'], axis=1)

In [59]:
# Calculate percentage of missing values for each column
nan_percentage = df_matches_dropped.isna().mean() * 100

# Print the percentage of missing values for each column
print(nan_percentage)

Unnamed: 0         0.000000
title_x            0.000000
imdbid             0.000000
id                 0.000000
rating_x           0.000000
rating_y          29.834254
certificate_x     29.834254
duration          29.834254
genre_x           29.834254
votes_x           29.834254
gross_income      29.834254
directors_id      29.834254
directors_name    29.834254
stars_id          29.834254
stars_name        29.834254
description       29.834254
runtime           44.513054
genre_y           44.513054
ratings           44.513054
director_x        44.513054
cast              44.513054
Description       44.513054
movie_link        44.513054
certificate_y     76.308092
runtime_min       76.308092
genre             76.308092
imdb_rating       76.308092
about             76.308092
director_y        76.308092
stars             76.308092
votes_y           76.308092
gross_earn        76.308092
Run Time          40.688983
Rating            40.688983
User Rating       40.688983
Generes           40

In [60]:
df_matches_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9231 entries, 0 to 9495
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      9231 non-null   int64  
 1   title_x         9231 non-null   object 
 2   imdbid          9231 non-null   float64
 3   id              9231 non-null   int64  
 4   rating_x        9231 non-null   int64  
 5   rating_y        6477 non-null   float64
 6   certificate_x   6477 non-null   object 
 7   duration        6477 non-null   object 
 8   genre_x         6477 non-null   object 
 9   votes_x         6477 non-null   object 
 10  gross_income    6477 non-null   object 
 11  directors_id    6477 non-null   object 
 12  directors_name  6477 non-null   object 
 13  stars_id        6477 non-null   object 
 14  stars_name      6477 non-null   object 
 15  description     6477 non-null   object 
 16  runtime         5122 non-null   object 
 17  genre_y         5122 non-null   o

In [ ]:
df_matches_dropped.to_csv('')